<a href="https://colab.research.google.com/github/rumman-adnan/Assignment-Sensors-Data/blob/main/AdvanceDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
# Load the training and test data
train_path = '/content/drive/MyDrive/Python work/sim_data/simu_20000_0.1_90_140_train.npy'
test_path = '/content/drive/MyDrive/Python work/sim_data/simu_10000_0.1_141_178_test.npy'
train_data = np.load(train_path)
test_data = np.load(test_path)

# Extract the sensor data and the target variables (S and D)
X_train = train_data[:, :1000]
y_train = train_data[:, -2:]
X_test = test_data[:, :1000]
y_test = test_data[:, -2:]

# Scale the sensor data (features)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
X_train_mean = np.mean(X_train, axis = 1)
X_train_mean = np.reshape(X_train_mean, (-1,1))
X_train_std = np.std(X_train, axis = 1)
X_train_std = np.reshape(X_train_std, (-1,1))
X_train = (X_train - X_train_mean)/X_train_std

X_test_mean = np.mean(X_test, axis = 1)
X_test_mean = np.reshape(X_test_mean, (-1,1))
X_test_std = np.std(X_test, axis = 1)
X_test_std = np.reshape(X_test_std, (-1,1))
X_test = (X_test - X_test_mean)/X_test_std

### Inception Time model

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping

# Reshape for Conv1D
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Inception module
def inception_module(input_tensor, filters):
    conv_1x1 = Conv1D(filters=filters, kernel_size=1, activation='relu')(input_tensor)
    conv_5x5 = Conv1D(filters=filters, kernel_size=5, padding='same', activation='relu')(input_tensor)
    conv_10x10 = Conv1D(filters=filters, kernel_size=10, padding='same', activation='relu')(input_tensor)
    max_pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(input_tensor)
    max_pool = Conv1D(filters=filters, kernel_size=1, activation='relu')(max_pool)
    inception = concatenate([conv_1x1, conv_5x5, conv_10x10, max_pool], axis=-1)
    return inception

# Build InceptionTime model
def build_inception_time(input_shape, num_output):
    input_layer = Input(shape=input_shape)
    x = inception_module(input_layer, 32)
    x = inception_module(x, 64)
    x = inception_module(x, 128)
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(num_output)(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create the model
input_shape = (X_train_scaled.shape[1], 1)
num_output = y_train.shape[1]
model = build_inception_time(input_shape, num_output)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# # Evaluate the model
# loss, mae = model.evaluate(X_test_scaled, y_test)
# print(f'Mean Absolute Error on Test Data: {mae}')
# Evaluate the model
y_predicted = model.predict(X_test_scaled)

# Calculate Mean Absolute Error between y_test and y_predicted
mae = np.mean(np.abs(y_test - y_predicted))

print(f'Mean Absolute Error on Test Data: {mae}')


Epoch 1/20
500/500 [==============================] - 36s 59ms/step - loss: 18.6869 - mean_absolute_error: 18.6869 - val_loss: 21.2178 - val_mean_absolute_error: 21.2178
Epoch 2/20
500/500 [==============================] - 30s 59ms/step - loss: 13.8880 - mean_absolute_error: 13.8880 - val_loss: 34.0906 - val_mean_absolute_error: 34.0906
Epoch 3/20
500/500 [==============================] - 30s 60ms/step - loss: 12.2645 - mean_absolute_error: 12.2645 - val_loss: 46.4770 - val_mean_absolute_error: 46.4770
Epoch 4/20
500/500 [==============================] - 30s 60ms/step - loss: 11.1924 - mean_absolute_error: 11.1924 - val_loss: 46.5492 - val_mean_absolute_error: 46.5492
Epoch 5/20
500/500 [==============================] - 30s 60ms/step - loss: 10.6523 - mean_absolute_error: 10.6523 - val_loss: 49.7262 - val_mean_absolute_error: 49.7262
Epoch 6/20
500/500 [==============================] - 30s 59ms/step - loss: 10.3474 - mean_absolute_error: 10.3474 - val_loss: 46.3695 - val_mean_abso

In [ ]:
from sklearn.metrics import mean_absolute_error
# Make predictions on the test set
predictions = model.predict(X_test_scaled)

# Separate the Systolic and Diastolic predictions
predictions_S = predictions[:, 0]
predictions_D = predictions[:, 1]

# Calculate MAE for both Systolic and Diastolic
mae_S = mean_absolute_error(y_test[:, 0], predictions_S)
mae_D = mean_absolute_error(y_test[:, 1], predictions_D)

print(f"Inception Time Model - Test MAE Systolic: {mae_S}, Diastolic: {mae_D}")

# Save the best model
# fcn_plus_model.save("best_fcn_plus_model.h5")

313/313 [==============================] - 6s 18ms/step
Inception Time Model - Test MAE Systolic: 61.16771869125366, Diastolic: 13.845622805786133


### TabTransformer Model

In [ ]:

X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Build TabTransformer model
def build_tab_transformer(input_shape, num_output):
    input_layer = Input(shape=input_shape)
    x = Embedding(input_dim=10000, output_dim=64)(input_layer)
    x = LayerNormalization(epsilon=1e-6)(x)
    transformer_out = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x + transformer_out)
    x = tf.keras.layers.Reshape((-1, 64))(x)  # Reshape the output to be 3D
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(num_output)(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create and compile the model
input_shape = (X_train_scaled.shape[1], 1)
num_output = y_train.shape[1]
model = build_tab_transformer(input_shape, num_output)
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

# Initialize variables for early stopping
best_mae_systolic = float('inf')
best_mae_diastolic = float('inf')
best_loss = float('inf')
patience = 10
counter = 0

# Custom training loop with early stopping
for epoch in range(1, 101):
    print(f"Epoch {epoch}")
    model.fit(X_train_scaled, y_train, batch_size=32, verbose=0)
    y_predicted = model.predict(X_test_scaled)
    loss = np.mean((y_test - y_predicted)**2)
    mae_systolic = np.mean(np.abs(y_test[:, 0] - y_predicted[:, 0]))
    mae_diastolic = np.mean(np.abs(y_test[:, 1] - y_predicted[:, 1]))
    print(f"Loss: {loss}, MAE Systolic: {mae_systolic}, MAE Diastolic: {mae_diastolic}")

    if mae_systolic < best_mae_systolic and mae_diastolic < best_mae_diastolic:
        best_mae_systolic = mae_systolic
        best_mae_diastolic = mae_diastolic
        best_loss = loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping due to lack of improvement.")
        break


Epoch 1
313/313 [==============================] - 14s 44ms/step
Loss: 933.1206082129501, MAE Systolic: 39.9994120803833, MAE Diastolic: 10.690308982849121
Epoch 2
313/313 [==============================] - 14s 45ms/step
Loss: 931.9349685156535, MAE Systolic: 39.889457096099854, MAE Diastolic: 10.77537036972046
Epoch 3
313/313 [==============================] - 14s 45ms/step
Loss: 1124.1249933541312, MAE Systolic: 44.74415615539551, MAE Diastolic: 10.261187128448487
Epoch 4
313/313 [==============================] - 14s 45ms/step
Loss: 1340.05868069504, MAE Systolic: 49.288928936767576, MAE Diastolic: 10.30135111618042
Epoch 5
313/313 [==============================] - 14s 45ms/step
Loss: 1381.8263191841083, MAE Systolic: 50.08697132949829, MAE Diastolic: 10.367178357696533
Epoch 6
313/313 [==============================] - 14s 45ms/step
Loss: 1409.897291146018, MAE Systolic: 50.175243676757816, MAE Diastolic: 11.17180807762146
Epoch 7


KeyboardInterrupt: ignored

In [ ]:
# Initialize variables to keep track of best MAE and loss
best_mae_systolic = float('inf')
best_mae_diastolic = float('inf')
best_loss = float('inf')
patience = 10  # Number of epochs to wait for improvement
counter = 0  # Counter to keep track of non-improvement epochs

for epoch in range(1, 101):  # Train for up to 100 epochs
    print(f"Epoch {epoch}")

    # Train the model for one epoch
    model.fit(X_train_scaled, y_train, batch_size=32, verbose=0)

    # Get the predicted values
    y_predicted = model.predict(X_test_scaled)

    # Calculate the loss and MAE between y_test and y_predicted
    loss = np.mean((y_test - y_predicted)**2)
    mae_systolic = np.mean(np.abs(y_test[:, 0] - y_predicted[:, 0]))
    mae_diastolic = np.mean(np.abs(y_test[:, 1] - y_predicted[:, 1]))

    print(f"Loss: {loss}, MAE Systolic: {mae_systolic}, MAE Diastolic: {mae_diastolic}")

    # Check for improvement
    if mae_systolic < best_mae_systolic and mae_diastolic < best_mae_diastolic:
        best_mae_systolic = mae_systolic
        best_mae_diastolic = mae_diastolic
        best_loss = loss
        counter = 0  # Reset counter
    else:
        counter += 1  # Increment counter

    # If MAE hasn't improved for 'patience' epochs, stop training
    if counter >= patience:
        print("Early stopping due to lack of improvement")
        break


### TCN Model

In [ ]:
pip install keras-tcn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.metrics import mean_absolute_error
from tcn import TCN

# X_train, X_test, y_train, y_test

# # 1. Load and scale features (sensor data), not output parameters
# train_data = np.load('simu_20000_0.1_90_140_train.npy')
# test_data = np.load('simu_10000_0.1_141_178_test.npy')

# X_train = train_data[:, :1000]
# y_train = train_data[:, -2:]
# X_test = test_data[:, :1000]
# y_test = test_data[:, -2:]

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Reshape for TCN
X_train_scaled = X_train_scaled.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test.shape[0], X_test.shape[1], 1)

# 2. Build the model using TCN
input_shape = (X_train_scaled.shape[1], 1)

inputs = Input(shape=input_shape)
x = TCN(nb_filters=64, kernel_size=6, dilations=[1, 2, 4, 8])(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(2)(x)  # 2 outputs (S and D)

TCN_model = Model(inputs=inputs, outputs=outputs)

# 3. Compile the model
TCN_model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

# 4. Train the model for 50 epochs with batch size 32
# TCN_model.fit(X_train_scaled, y_train, epochs=80, batch_size=32, validation_split=0.2, Shuffle=False)
TCN_model.fit(X_train_scaled, y_train, epochs=80, batch_size=32, validation_split=0.2, shuffle=False)


# 5. Evaluate the model
loss, mae = TCN_model.evaluate(X_test_scaled, y_test, batch_size=32)
print(f'Test Loss After 150 Epochs: {loss}, Test MAE After 150 Epochs: {mae}')

# y_pred_systolic = model.predict(X_test_sys) #.squeeze()
# # mae_systolic = MeanAbsoluteError()
# mae_value = mean_absolute_error(y_test_sys, y_pred_systolic)

# print(f'MAE between y_pred_systolic and y_test_systolic After 150 Epochs: {mae_value}')


# Make predictions on the test set
predictions = TCN_model.predict(X_test_scaled)

# Separate the Systolic and Diastolic predictions
predictions_S = predictions[:, 0]
predictions_D = predictions[:, 1]

# Calculate MAE for both Systolic and Diastolic
mae_S = mean_absolute_error(y_test[:, 0], predictions_S)
mae_D = mean_absolute_error(y_test[:, 1], predictions_D)

print(f"TCN Model - Test MAE Systolic: {mae_S}, Diastolic: {mae_D}")




Epoch 1/80
500/500 [==============================] - 8s 8ms/step - loss: 14.5316 - mean_absolute_error: 14.5316 - val_loss: 10.9926 - val_mean_absolute_error: 10.9926
Epoch 2/80
500/500 [==============================] - 3s 7ms/step - loss: 8.7842 - mean_absolute_error: 8.7842 - val_loss: 7.5681 - val_mean_absolute_error: 7.5681
Epoch 3/80
500/500 [==============================] - 3s 7ms/step - loss: 7.2836 - mean_absolute_error: 7.2836 - val_loss: 6.2774 - val_mean_absolute_error: 6.2774
Epoch 4/80
500/500 [==============================] - 3s 7ms/step - loss: 6.6424 - mean_absolute_error: 6.6424 - val_loss: 6.4812 - val_mean_absolute_error: 6.4812
Epoch 5/80
500/500 [==============================] - 3s 7ms/step - loss: 6.3095 - mean_absolute_error: 6.3095 - val_loss: 5.2650 - val_mean_absolute_error: 5.2650
Epoch 6/80
500/500 [==============================] - 3s 7ms/step - loss: 5.4766 - mean_absolute_error: 5.4766 - val_loss: 7.6745 - val_mean_absolute_error: 7.6745
Epoch 7/80
5

### TCN Model
- Shuffle (true)
- Results are below

In [ ]:
# print(f'Test Loss: {loss}, Test MAE: {mae}')
predictions = model.predict(X_test_scaled)

# Separate the Systolic and Diastolic predictions
predictions_S = predictions[:, 0]
predictions_D = predictions[:, 1]

# Calculate MAE for both Systolic and Diastolic
mae_S = mean_absolute_error(y_test[:, 0], predictions_S)
mae_D = mean_absolute_error(y_test[:, 1], predictions_D)

print(f"TCN Model - Test MAE Systolic: {mae_S}, Diastolic: {mae_D}")

313/313 [==============================] - 1s 2ms/step
TCN Model - Test MAE Systolic: 21.798146901702882, Diastolic: 9.598864441680908


### TCN with Z-Transform Scaling

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tcn import TCN
from tensorflow.keras.losses import MeanAbsoluteError

# Load data
# train_data = np.load('simu_20000_0.1_90_140_train.npy')
# test_data = np.load('simu_10000_0.1_141_178_test.npy')

X_train = train_data[:, :1000]
y_train_sys = train_data[:, -2]  # Only systolic BP
X_test = test_data[:, :1000]
y_test_sys = test_data[:, -2]  # Only systolic BP

# Scale X_train_sys
X_train_mean = np.mean(X_train, axis=1)
X_train_mean = np.reshape(X_train_mean, (-1, 1))
X_train_std = np.std(X_train, axis=1)
X_train_std = np.reshape(X_train_std, (-1, 1))
X_train_sys = (X_train - X_train_mean) / X_train_std

X_test_mean = np.mean(X_test, axis=1)
X_test_mean = np.reshape(X_test_mean, (-1, 1))
X_test_std = np.std(X_test, axis=1)
X_test_std = np.reshape(X_test_std, (-1, 1))
X_test_sys = (X_test - X_test_mean) / X_test_std

# Reshape for TCN
X_train_sys = X_train_sys.reshape(X_train_sys.shape[0], X_train_sys.shape[1], 1)
X_test_sys = X_test_sys.reshape(X_test_sys.shape[0], X_test_sys.shape[1], 1)

# Build the model
input_shape = (X_train_sys.shape[1], 1)
inputs = Input(shape=input_shape)
x = TCN(nb_filters=64, kernel_size=6, dilations=[1, 2, 4, 8])(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1)(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=[MeanAbsoluteError()])

# Train the model for 70 epochs
model.fit(X_train_sys, y_train_sys, epochs=80, batch_size=32, validation_split=0.2, shuffle=False)

# Evaluate the model
loss, mae = model.evaluate(X_test_sys, y_test_sys, batch_size=32)
print(f'Test Loss: {loss}, Test MAE: {mae}')

y_pred_systolic = model.predict(X_test_sys) #.squeeze()
# mae_systolic = MeanAbsoluteError()
mae_value = mean_absolute_error(y_test_sys, y_pred_systolic)

print(f'MAE between y_pred_systolic and y_test_systolic After 150 Epochs: {mae_value}')

Epoch 1/80
499/500 [============================>.] - ETA: 0s - loss: 16.3257 - mean_absolute_error: 16.3257

KeyboardInterrupt: ignored

In [ ]:
# Calculate simple MAE between y_pred_systolic and y_test_systolic
y_pred_systolic = model.predict(X_test_sys) #.squeeze()
# mae_systolic = MeanAbsoluteError()
mae_value = mean_absolute_error(y_test_sys, y_pred_systolic)
print(f'MAE between y_pred_systolic and y_test_systolic: {mae_value}')

313/313 [==============================] - 1s 2ms/step
MAE between y_pred_systolic and y_test_systolic: 17.825097306823732


In [ ]:
# Continue training the existing model for an additional 80 epochs
model.fit(X_train_sys, y_train, epochs=80, batch_size=32, validation_split=0.2, shuffle=False)

# Evaluate the model again after the additional 80 epochs
# loss, mae = model.evaluate(X_test, y_test, batch_size=32)
print(f'Test Loss After 150 Epochs: {loss}, Test MAE After 150 Epochs: {mae}')

# Calculate simple MAE between y_pred_systolic and y_test_systolic after the additional 80 epochs
y_pred_systolic = model.predict(X_test_sys) #.squeeze()
# mae_systolic = MeanAbsoluteError()
mae_value = mean_absolute_error(y_test_sys, y_pred_systolic)

print(f'MAE between y_pred_systolic and y_test_systolic After 150 Epochs: {mae_value}')


Epoch 1/80
500/500 [==============================] - 7s 10ms/step - loss: 17.6925 - mean_absolute_error: 17.6925 - val_loss: 17.6075 - val_mean_absolute_error: 17.6075
Epoch 2/80
500/500 [==============================] - 4s 9ms/step - loss: 17.6811 - mean_absolute_error: 17.6811 - val_loss: 17.6327 - val_mean_absolute_error: 17.6327
Epoch 3/80
500/500 [==============================] - 4s 9ms/step - loss: 17.6866 - mean_absolute_error: 17.6866 - val_loss: 17.5951 - val_mean_absolute_error: 17.5951
Epoch 4/80
500/500 [==============================] - 4s 8ms/step - loss: 17.6930 - mean_absolute_error: 17.6930 - val_loss: 17.5958 - val_mean_absolute_error: 17.5958
Epoch 5/80
500/500 [==============================] - 4s 9ms/step - loss: 17.6987 - mean_absolute_error: 17.6987 - val_loss: 17.6384 - val_mean_absolute_error: 17.6384
Epoch 6/80
500/500 [==============================] - 4s 9ms/step - loss: 17.7113 - mean_absolute_error: 17.7113 - val_loss: 17.7238 - val_mean_absolute_error:

InvalidArgumentError: ignored

In [ ]:
y_pred_systolic = model.predict(X_test_sys) #.squeeze()
# mae_systolic = MeanAbsoluteError()
mae_value = mean_absolute_error(y_test_sys, y_pred_systolic)

print(f'MAE between y_pred_systolic and y_test_systolic After 150 Epochs: {mae_value}')

313/313 [==============================] - 1s 2ms/step
MAE between y_pred_systolic and y_test_systolic After 150 Epochs: 52.40868386917114


In [ ]:
loss, mae = model.evaluate(X_test_sys, y_test_sys, batch_size=32)
print(f'Test Loss After 150 Epochs: {loss}, Test MAE After 150 Epochs: {mae}')



313/313 [==============================] - 1s 3ms/step - loss: 52.4087 - mean_absolute_error: 52.4060
Test Loss After 150 Epochs: 52.40868377685547, Test MAE After 150 Epochs: 52.406005859375
